#### Imports

In [1]:
import requests
from joblib import Parallel, delayed
import pandas as pd
pd.set_option('display.max_colwidth', -1)
import pickle
from os import path
import networkx as nx
import matplotlib.pyplot as plt

#### Constants

In [2]:
API = "https://explorer.helium.foundation/api"

#### Helpers

In [3]:
def dashed(name):
    return "-".join(name.lower().split(" "))

def get_witnesses(hname, b58):
    r = requests.get("{}/witnesses/{}".format(API, b58))
    assert(r.status_code == 200)
    witnesses = r.json()['data']
    if len(witnesses) > 0:
        witnesses = [dashed(w['name']) for w in witnesses]
    else:
        witnesses = []
    return {"name": dashed(hname), "witnesses": witnesses}

#### Hotspots

In [4]:
hr = requests.get("{}/hotspots".format(API))
assert(hr.status_code == 200)
hotspots = hr.json()['data']

#### Witnesses file pickled 3033 hotspots @ height 198410

In [5]:
# check if we have witnesses locally
witness_pickle = 'files/witnesses.pickle'
if path.exists(witness_pickle):
    with open(witness_pickle, 'rb') as handle:
        witness_list = pickle.load(handle)
else:
    # make the witness fetching marginally faster by parallelizing requests
    witness_list = Parallel(n_jobs=8)(delayed(get_witnesses)(n, a) for (n, a) in [(x['name'], x['address']) for x in hotspots])
    with open(witness_pickle, 'wb') as handle:
        pickle.dump(witness_list, handle, protocol=pickle.HIGHEST_PROTOCOL)

witnesses = {}
for d in witness_list:
    witnesses[d['name']] = d['witnesses']

In [6]:
G = nx.Graph()
G.add_nodes_from(witnesses.keys())
for k, v in witnesses.items():
    G.add_edges_from([(k, t) for t in v])

##### Is the network connected?

In [7]:
nx.is_connected(G)

False

##### What is the cluster average?

In [8]:
nx.algorithms.cluster.average_clustering(G)

0.3990007550866049

#### How many total components? For example: a graph with three compents:
<img src="files/3-component-graph.png" style="width:400px">

In [9]:
nx.number_connected_components(G)

1276

In [10]:
len(nx.node_connected_component(G, 'muscular-slate-meerkat'))

351

In [11]:
len(nx.node_connected_component(G, 'tall-blonde-condor'))

3

In [12]:
len(nx.node_connected_component(G, 'huge-glossy-robin'))

351

In [13]:
list(nx.neighbors(G, 'huge-glossy-robin'))

['sharp-jetblack-dove']

In [14]:
list(nx.neighbors(G, 'tall-blonde-condor'))

['proud-clay-aardvark', 'short-sky-panther']

In [15]:
list(nx.neighbors(G, 'lone-violet-orca'))

['spicy-honey-hawk',
 'zealous-cream-swallow',
 'muscular-slate-meerkat',
 'rare-burlap-lizard']

In [16]:
len(nx.node_connected_component(G, 'lone-violet-orca'))

351

In [17]:
cc = list(nx.connected_components(G))
scc = sorted(cc, key=len, reverse=True)
print('Total connected components (sub-graphs): {}'.format(len(scc)))

Total connected components (sub-graphs): 1276


In [18]:
print('Top 50 biggest sub graphs: \n{}'.format([len(i) for i in scc[:50]]))

Top 50 biggest sub graphs: 
[351, 286, 154, 111, 94, 82, 62, 55, 35, 33, 31, 27, 26, 22, 22, 20, 20, 20, 17, 14, 13, 12, 12, 10, 10, 10, 10, 10, 9, 7, 6, 6, 6, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3]
